In [2]:
import tensorflow as tf
import os
import numpy as np
import cv2 as cv

import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

import matplotlib.pyplot as plt

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
VALIDATION_IMAGES = 200
TRAIN_IMAGES = 2000

IMAGE_SIZE = 224
GRID_SIZE = 5
BOX_SIZE = 1 #originally 2 boxes -> choose the one with the highest iou => we will not do in here
NUM_CLASS = 2
BATCH_SIZE = 128

In [5]:
classesNum = {'dog': 0, 'cat': 1}
def convertFunction(folder, name, file):
    path = folder + '/' + name
    path = os.path.normpath(path)
    tree = ET.parse(path)
    root = tree.getroot()

    file.write(path.replace('xml', 'png'))
    for obj in root.iter('object'):
        difficult = obj.find('difficult').text
        className = obj.find('name').text
        if className not in classesNum.keys() or int(difficult) == 1:
            continue

        #get bounding box
        box = ( int(obj.find('bndbox').find('xmin').text),
                int(obj.find('bndbox').find('ymin').text),
                int(obj.find('bndbox').find('xmax').text),
                int(obj.find('bndbox').find('ymax').text))

        id = list(classesNum.keys()).index(className)

        #write to file
        file.write(' ' + ','.join([str(a) for a in box]) + ',' + str(id))
    file.write('\n')

with open(os.path.join('%s.txt' % ('annotations')) , 'w') as f:
    for file in os.listdir('annotations'):
        if file.endswith('.xml'):
            convertFunction(folder = 'annotations', name=str(file), file=f)

In [6]:
train_datasets = []
with open(os.path.normpath('annotations.txt'), 'r') as f:
    train_datasets = f.readlines()

In [7]:
len(train_datasets)

3686

In [8]:
val_datasets = train_datasets[:VALIDATION_IMAGES]
train_datasets = train_datasets[VALIDATION_IMAGES : VALIDATION_IMAGES + TRAIN_IMAGES]

In [9]:
print('Train images: ', len(train_datasets), 'Val images: ', len(val_datasets))

Train images:  2000 Val images:  200


In [10]:
def annotationConverting(dataset):
    X, Y = [], []
    for item in dataset:
        item = item.replace("\n", "").split(" ")
        X.append(item[0])
        arr = []
        for i in range(1, len(item)):
            arr.append(item[i])
        Y.append(arr)
    return X,Y


In [11]:
X_train, Y_train = annotationConverting(train_datasets)
X_val, Y_val = annotationConverting(val_datasets)

In [12]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, images, labels, shuffle=False):
        self.images = images
        self.labels = labels
        self.shuffle = shuffle
    
    def __len__(self):
        return (np.ceil(len(self.images) / int(BATCH_SIZE))).astype(int)
    
    def __getitem__(self, idx):
        batch_x = self.images[idx * BATCH_SIZE : (idx + 1) * BATCH_SIZE] # images path
        batch_y = self.labels[idx * BATCH_SIZE : (idx + 1) * BATCH_SIZE] # raw label

        train_image, train_label = [], []
        for i in range(0, len(batch_x)):
            img_path, label = batch_x[i], batch_y[i]
            image, label_matrix = self.read(img_path, label) #actual image array (IMAGE_SIZE, IMAGE_SIZE, 3) (GRID_SIZE, GRID_SIZE, 5 * BOX_SIZE + CLASS) 
            train_image.append(image)
            train_label.append(label_matrix)
        
        if self.shuffle:
            indices = tf.range(start=0, limit=tf.shape(train_image)[0], dtype=tf.int32)
            idx = tf.random.shuffle(indices)
            train_image = tf.gather(train_image, idx)
            train_label = tf.gather(train_label, idx)

        return np.array(train_image, dtype=np.float32), np.array(train_label, dtype=np.float32)
    
    def read(self, img_path, label):
        image = cv.imread(img_path)
        h, w = image.shape[0:2]
        image = cv.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
        image = image / 255.

        label_matrix = np.zeros([GRID_SIZE, GRID_SIZE, 5 * BOX_SIZE + NUM_CLASS])
        for l in label:
            l = l.split(',')
            l = np.array(l, dtype=int)

            xmin, ymin, xmax, ymax = l[0] / w, l[1] / h, l[2] / w, l[3] / h # [0, 1]
            
            x, y = (xmin + xmax) / 2, (ymin + ymax) / 2
            w, h = xmax - xmin, ymax - ymin

            #convert x, y relative to the cell
            i, j = int(GRID_SIZE * y), int(GRID_SIZE * x)
            x = GRID_SIZE * x - j # 7 * [0,1] = [0, 7] e.g: 6.43 - int(6.43) = 0.43 => relative to the cell
            y = GRID_SIZE * y - i

            if l[4] == 0:
                label_matrix[i, j] = [x, y, w, h, 1, 1, 0]
            if l[4] == 1:
                label_matrix[i, j] = [x, y, w, h, 1, 0, 1]
        return image, label_matrix

In [13]:
trainingDataGenerator = DataGenerator(X_train, Y_train)
validationDataGenerator = DataGenerator(X_val, Y_val)

x_train, y_train = trainingDataGenerator.__getitem__(0)

In [14]:
classArray = list(classesNum.keys())
print(classArray)

['dog', 'cat']


In [15]:
def TestPrint(image, label):
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            if label[i][j][4] > 0.5:
                print(label[i][j])
                x, y, w, h = label[i][j][:4]

                xmax = int(((x + j) / GRID_SIZE * IMAGE_SIZE) + (w * IMAGE_SIZE) / 2)
                xmin = int(((x + j) / GRID_SIZE * IMAGE_SIZE) - (w * IMAGE_SIZE) / 2)
                ymax = int(((y + i) / GRID_SIZE * IMAGE_SIZE) + (h * IMAGE_SIZE) / 2)
                ymin = int(((y + i) / GRID_SIZE * IMAGE_SIZE) - (h * IMAGE_SIZE) / 2)

                className = classArray[tf.argmax(label[i][j][5:], axis=-1)]
                cv.putText(image, className, (xmin, ymax + 10), cv.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 255, 255))
                cv.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 255, 255), 1)
    #image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    cv.imshow('Visualize', image)
    cv.waitKey(0)
    cv.destroyWindow('Visualize')


In [16]:
x_train, y_train = trainingDataGenerator.__getitem__(0)
idx = 6
TestPrint(x_train[idx], y_train[idx])

In [ ]:
class YoloActivation(tf.keras.layers.Layer):
    def call(self, inputs):
        classes = tf.nn.softmax(inputs[..., 5:], axis=-1)
        coordinates = tf.sigmoid(inputs[..., :5])
        return tf.concat([coordinates, classes], axis=-1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, MaxPool2D, BatchNormalization

lrelu = tf.keras.layers.LeakyReLU(alpha=0.1)

featureExtractor = Sequential()
featureExtractor.add(tf.keras.applications.efficientnet_v2.EfficientNetV2M(
    include_top = False,
    weights = 'imagenet',
    input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3),
))

featureExtractor.trainable = False

model = Sequential()
model.add(featureExtractor)
model.add(MaxPool2D((2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(1024, activation=lrelu))
model.add(Dropout(0.5))
model.add(Dense(512, activation=lrelu))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(GRID_SIZE * GRID_SIZE * (BOX_SIZE * 5 + NUM_CLASS))) #total nodes we need -> reshape them into the grid (GRID_SIZE, GRID_SIZE, BOX_SIZE * 5 + NUM_CLASS)
model.add(Reshape((GRID_SIZE, GRID_SIZE, BOX_SIZE * 5 + NUM_CLASS))) # linear activation function (-inf, inf) -> takes longer to converge and not that good
model.add(YoloActivation()) #convert last two class probability into a softmax outputs

model.summary()

82420632/82420632 [==============================] - 4s 0us/step
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 7, 7, 1280)        20331360  
                                                                 
 max_pooling2d (MaxPooling2D  (None, 3, 3, 1280)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 3, 3, 1280)       5120      
 ormalization)                                                   
                                                                 
 flatten (Flatten)           (None, 11520)             0         
                                                                 
 dense (Dense)               (None, 1024)              11797504  
                                                       

In [ ]:
#YOLO LOSS !!!
def yoloLoss(y_true, y_pred):
    coordLoss = CoordLoss(y_true, y_pred)
    confidenceLoss = ConfidenceLoss(y_true, y_pred)
    classLoss = ClassLoss(y_true, y_pred)

    return 25 * coordLoss + 15 * confidenceLoss + 5 * classLoss

def CoordLoss(y_true, y_pred):
    #find if it exist an object in the grid
    existsObject = tf.expand_dims(y_true[..., 4], -1)

    xy_pred = existsObject * y_pred[..., 0:2]
    xy_true = existsObject * y_true[..., 0:2]

    wh_pred = existsObject * tf.math.sign(y_pred[..., 2:4]) * tf.sqrt(tf.math.abs(y_pred[..., 2:4])) #if it's linear (-inf, inf)
    wh_true = existsObject * tf.sqrt(y_true[..., 2:4])

    coordLoss = tf.reduce_sum(tf.math.square(wh_pred - wh_true))
    coordLoss += tf.reduce_sum(tf.math.square(xy_pred - xy_true))

    return coordLoss / tf.cast(tf.math.count_nonzero(existsObject), dtype=tf.float32) #mean, but it's fine if we don't

def ConfidenceLoss(y_true, y_pred):
    existsObject = tf.expand_dims(y_true[..., 4], -1)

    confidenceLoss = tf.reduce_sum(tf.math.square(existsObject * (y_true[..., 4:5] - y_pred[..., 4:5])))
    confidenceLoss += 0.5*tf.reduce_sum(tf.math.square((1 - existsObject) * (y_true[..., 4:5] - y_pred[..., 4:5])))

    return confidenceLoss / tf.cast(tf.math.count_nonzero(existsObject), dtype=tf.float32) #mean, but it's fine if we don't

def ClassLoss(y_true, y_pred):
    existsObject = tf.expand_dims(y_true[..., 4], -1)

    classLoss = tf.reduce_sum(tf.math.square(existsObject * (y_true[..., 5:] - y_pred[..., 5:])))
    return classLoss / tf.cast(tf.math.count_nonzero(existsObject), dtype=tf.float32) #mean, but it's fine if we don't


In [ ]:
model.compile(loss = yoloLoss, optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=[CoordLoss, ConfidenceLoss, ClassLoss]) 

In [ ]:
model.fit(  x = trainingDataGenerator,
            validation_data = (validationDataGenerator),
            epochs = 300,
            workers = 8,
            validation_freq = 5)

Epoch 1/300


In [ ]:
for i in range(9):
    image = cv.imread('annotations/Cats_Test20' + str(i) + '.png')
    image = cv.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    image = image / 255.
    TestPrint(image, model.predict(np.expand_dims(image, 0))[0])

1/1 [==============================] - 0s 47ms/step
[0.2942742  0.5018323  0.4791737  0.58642685 0.99996674 0.99850523
 0.00149476]
1/1 [==============================] - 0s 52ms/step
[3.0476224e-01 8.3730859e-01 1.5449563e-01 1.4743602e-01 9.9920017e-01
 9.9975306e-01 2.4693698e-04]
1/1 [==============================] - 0s 62ms/step
[7.8192466e-01 5.7377201e-01 2.8494388e-01 3.1162944e-01 9.9926645e-01
 9.9996006e-01 3.9913368e-05]
1/1 [==============================] - 0s 53ms/step
[1.9740531e-01 1.2624970e-01 3.1917673e-01 3.8819090e-01 9.9303764e-01
 8.3120285e-05 9.9991691e-01]
1/1 [==============================] - 0s 59ms/step
[0.5699612  0.20659435 0.33492556 0.34023908 0.99761957 0.9951994
 0.00480065]
1/1 [==============================] - 0s 50ms/step
[0.44433254 0.57880026 0.326442   0.3963741  0.9999447  0.0014199
 0.9985801 ]
1/1 [==============================] - 0s 53ms/step
[8.8392013e-01 7.8798074e-01 3.4697822e-01 6.4350444e-01 9.9999905e-01
 9.9965823e-01 3.4172251

In [ ]:
model.save('yolov1')

INFO:tensorflow:Assets written to: yolov1\assets


INFO:tensorflow:Assets written to: yolov1\assets
